## Загрузка данных

In [2]:
!pip install nltk
!python -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import os
import re
import spacy
import wandb
import numpy as np
import pandas as pd
from datasets import load_dataset, DatasetDict

from dataclasses import dataclass
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import chain
from typing import List, Dict, Tuple
from collections import Counter, defaultdict

## WandB

In [4]:
import wandb 
from kaggle_secrets import UserSecretsClient

In [5]:
def init_wandb(project_name="translation-transformer", config=None):
    try:
        user_secrets = UserSecretsClient()
        
        wandb_api_key = user_secrets.get_secret("wandb")
        os.environ['WANDB_API_KEY'] = wandb_api_key
        
        wandb.login(key=wandb_api_key)
        
        run = wandb.init(
            project=project_name,
            config=config
        )
        
        print("✅ W&B successfully initialized")
        return run
    
    except Exception as e:
        print(f"❌ Error initializing W&B: {str(e)}")

In [6]:
config = {
    "architecture": "transformer",
    "subset_size": 100,
    "epochs": 5,
    "batch_size": 32,
    "learning_rate": 0.0001,
    "eval_steps": 50,
    "d_model": 512,
    "num_heads": 8,
    "num_layers": 6,
}

run = init_wandb(config=config)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: eva-koroleva (ml-samurai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


✅ W&B successfully initialized


## Dataset

In [7]:
def load_translation_dataset():
    print("Loading Tatoeba en-ru...")
    try:
        dataset = load_dataset("Helsinki-NLP/tatoeba", lang1="en", lang2="ru", trust_remote_code=True)
        
    except Exception as e:
        print(f"Error while loading dataset: {e}")
        raise
    
    print("\nDataset structure:")
    print(dataset)
    
    print("\nData sample:")
    for i in range(2):
        print(f"EN: {dataset['train'][i]['translation']['en']}")
        print(f"RU: {dataset['train'][i]['translation']['ru']}\n")

    return dataset

In [8]:
dataset = load_translation_dataset()

Loading Tatoeba en-ru...


README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

tatoeba.py:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]


Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

Data sample:
EN: For once in my life I'm doing a good deed... And it is useless.
RU: Один раз в жизни я делаю хорошее дело... И оно бесполезно.

EN: Let's try something.
RU: Давайте что-нибудь попробуем!



In [9]:
from transformers import AutoTokenizer
import numpy as np

def prepare_data_with_hf(
    dataset, 
    model_name: str = "Helsinki-NLP/opus-mt-en-ru", 
    max_length: int = 128, 
    batch_size: int = 32
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    special_tokens = {
        'bos_token': '<s>',
        'eos_token': '</s>',
        'pad_token': '<pad>',
        'unk_token': '<unk>'
    }
    tokenizer.add_special_tokens(special_tokens)
    
    tokenizer.bos_token = '<s>'
    tokenizer.eos_token = '</s>'
    tokenizer.pad_token = '<pad>'
    tokenizer.unk_token = '<unk>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<s>')
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('</s>')
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<pad>')

    def preprocess_function(examples):
        source_texts = [
            f"{tokenizer.bos_token} {item['en']}" 
            for item in examples['translation']
        ]
        
        target_texts = [
            f"{tokenizer.bos_token} {item['ru']} {tokenizer.eos_token}"
            for item in examples['translation']
        ]

        source_encoding = tokenizer(
            source_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np',
            add_special_tokens=False 
        )
        
        target_encoding = tokenizer(
            target_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np',
            add_special_tokens=False
        )

        source_ids = source_encoding['input_ids']
        source_ids[:, 0] = tokenizer.bos_token_id 
        
        target_ids = target_encoding['input_ids']
        target_ids[:, 0] = tokenizer.bos_token_id 
        
        target_attention_mask = target_encoding['attention_mask']
        seq_lens = np.argmin(target_attention_mask, axis=1)
        seq_lens[seq_lens == 0] = target_attention_mask.shape[1]
        
        for i, pos in enumerate(seq_lens):
            if pos < max_length:
                target_ids[i, pos-1] = tokenizer.eos_token_id
            else:
                target_ids[i, -1] = tokenizer.eos_token_id

        return {
            'input_ids': source_ids,
            'attention_mask': source_encoding['attention_mask'],
            'labels': target_ids,
            'decoder_attention_mask': target_encoding['attention_mask']
        }
    
    processed_dataset = dataset['train'].map(
        preprocess_function,
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset['train'].column_names
    )
    
    return processed_dataset, tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
processed_data, hf_tokenizer = prepare_data_with_hf(dataset)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/523656 [00:00<?, ? examples/s]

In [11]:
sample = processed_data[0]
print("Source:", hf_tokenizer.decode(sample['input_ids']))
print("Target:", hf_tokenizer.decode(sample['labels']))

Source: <s>  For once in my life I'm doing a good deed... And it is useless.<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Target: <s> Один раз в жизни я делаю хорошее дело... И оно бесполезно.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

## Архитектура модели

In [12]:
%%capture
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

In [13]:
import os
import time
import math
import copy
import spacy
import GPUtil
import pandas as pd
from typing import *
from itertools import chain

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset

import altair as alt
from altair import Chart

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach()
        return self.dropout(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q: [batch_size, num_heads, seq_len, head_dim]
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        attn_probs = F.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        output = self.W_o(attn_output)
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, mask=None):
        # Self attention
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Self attention (маскированное)
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Cross attention (с выходом энкодера)
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, num_layers=6):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0.1)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads) for _ in range(num_layers)])
        
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
            
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Проверка размерностей входных данных
        batch_size = src.size(0)
        src_seq_len = src.size(1)
        tgt_seq_len = tgt.size(1)
        
        # Энкодинг
        src_emb = self.positional_encoding(self.encoder_embedding(src))
        enc_output = src_emb
        
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)
        
        # Декодинг
        tgt_emb = self.positional_encoding(self.decoder_embedding(tgt))
        dec_output = tgt_emb
        
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        
        # Финальный слой
        output = self.fc_out(dec_output)
        return output

In [15]:
def test_transformer():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    batch_size = 2
    seq_len = 10
    d_model = 512
    num_heads = 8
    src_vocab_size = 100
    tgt_vocab_size = 100
    num_layers = 2

    src = torch.randint(0, src_vocab_size, (batch_size, seq_len)).to(device)
    tgt = torch.randint(0, tgt_vocab_size, (batch_size, seq_len)).to(device)
    
    src_mask = torch.ones(batch_size, 1, 1, seq_len).to(device)
    tgt_mask = torch.tril(torch.ones(seq_len, seq_len)).expand(batch_size, 1, seq_len, seq_len).to(device)

    transformer = Transformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    ).to(device)

    print("="*50)
    print("1. Тест Positional Encoding")
    pe = PositionalEncoding(d_model, dropout=0.1).to(device)
    x = torch.randn(1, seq_len, d_model).to(device)
    print(f"До PE: mean={x.mean().item():.4f}, std={x.std().item():.4f}")
    x_pe = pe(x)
    print(f"После PE: mean={x_pe.mean().item():.4f}, std={x_pe.std().item():.4f}")
    print(f"Форма PE: {x_pe.shape} (должна быть [1, {seq_len}, {d_model}])")
    
    print("\n2. Тест Multi-Head Attention")
    mha = MultiHeadAttention(d_model, num_heads).to(device)
    q = k = v = torch.randn(batch_size, seq_len, d_model).to(device)
    attn_output = mha(q, k, v)
    print(f"Форма выхода внимания: {attn_output.shape} (должна быть {q.shape})")
    print(f"Максимальное значение: {attn_output.max().item():.4f}")
    print(f"Минимальное значение: {attn_output.min().item():.4f}")

    print("\n3. Тест Encoder Layer")
    encoder_layer = EncoderLayer(d_model, num_heads).to(device)
    enc_input = torch.randn(batch_size, seq_len, d_model).to(device)
    enc_output = encoder_layer(enc_input)
    print(f"Форма выхода энкодера: {enc_output.shape} (должна быть {enc_input.shape})")
    print(f"Изменение данных: {torch.allclose(enc_input, enc_output, atol=1e-4)} (должно быть False)")

    print("\n4. Тест Decoder Layer")
    decoder_layer = DecoderLayer(d_model, num_heads).to(device)
    dec_input = torch.randn(batch_size, seq_len, d_model).to(device)
    dec_output = decoder_layer(dec_input, enc_output, src_mask, tgt_mask)
    print(f"Форма выхода декодера: {dec_output.shape} (должна быть {dec_input.shape})")
    print(f"Норма выходных данных: {dec_output.norm().item():.4f}")

    print("\n5. Полный тест Transformer")
    print("Входные данные:")
    print(f"src: {src.shape} (max={src.max().item()}, min={src.min().item()})")
    print(f"tgt: {tgt.shape} (max={tgt.max().item()}, min={tgt.min().item()})")
    
    output = transformer(src, tgt, src_mask, tgt_mask)
    print("\nПроверка формы выхода:")
    print(f"Ожидаемая форма: ({batch_size}, {seq_len}, {tgt_vocab_size})")
    print(f"Реальная форма:   {output.shape}")
    
    print("\nПроверка градиентов:")
    dummy_loss = output.sum()
    dummy_loss.backward()
    has_gradients = any(p.grad is not None for p in transformer.parameters())
    print(f"Градиенты вычислены: {has_gradients} (должно быть True)")

    print("\n6. Проверка параметров модели:")
    total_params = sum(p.numel() for p in transformer.parameters())
    print(f"Всего параметров: {total_params}")
    print(f"Параметры энкодера: {sum(p.numel() for p in transformer.encoder_embedding.parameters())}")
    print(f"Параметры декодера: {sum(p.numel() for p in transformer.decoder_embedding.parameters())}")

    print("\nТест завершен!")

In [16]:
test_transformer()

Using device: cuda
1. Тест Positional Encoding
До PE: mean=-0.0316, std=1.0065
После PE: mean=0.4401, std=1.2013
Форма PE: torch.Size([1, 10, 512]) (должна быть [1, 10, 512])

2. Тест Multi-Head Attention
Форма выхода внимания: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Максимальное значение: 0.3949
Минимальное значение: -0.4440

3. Тест Encoder Layer
Форма выхода энкодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Изменение данных: False (должно быть False)

4. Тест Decoder Layer
Форма выхода декодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Норма выходных данных: 101.1924

5. Полный тест Transformer
Входные данные:
src: torch.Size([2, 10]) (max=95, min=6)
tgt: torch.Size([2, 10]) (max=99, min=10)

Проверка формы выхода:
Ожидаемая форма: (2, 10, 100)
Реальная форма:   torch.Size([2, 10, 100])

Проверка градиентов:
Градиенты вычислены: True (должно быть True)

6. Проверка параметров модели:
Всего параметров: 14866532
Па

In [17]:
def translate_sentence(model, tokenizer, sentence, device, max_length=128):
    model.eval()
    
    inputs = tokenizer(
        f"{tokenizer.bos_token} {sentence}",
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)
    
    decoder_input = torch.tensor([[tokenizer.bos_token_id]], device=device)
    
    with torch.no_grad():
        for _ in range(max_length):
            src_mask = (inputs['input_ids'] != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(decoder_input.size(1), decoder_input.size(1), device=device)).bool()
            
            output = model(
                inputs['input_ids'],
                decoder_input,
                src_mask,
                tgt_mask.unsqueeze(0).unsqueeze(0)
            )
            
            next_token = output[:, -1].argmax(-1)
            decoder_input = torch.cat([decoder_input, next_token.unsqueeze(1)], dim=1)
            
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    return tokenizer.decode(decoder_input[0], skip_special_tokens=True)

## Тестирование

In [18]:
import torch
print(torch.__version__)  # Должно быть >= 1.11.0

2.5.1+cu121


In [19]:
def test_tokenization(tokenizer, test_sentence="Hello, how are you?"):
    print("="*50)
    print("1. Тестирование токенизации и специальных токенов")
    
    print(f"BOS: {tokenizer.bos_token} (id: {tokenizer.bos_token_id})")
    print(f"EOS: {tokenizer.eos_token} (id: {tokenizer.eos_token_id})")
    
    # Явное добавление BOS/EOS как в основном пайплайне
    processed_text = f"{tokenizer.bos_token} {test_sentence} {tokenizer.eos_token}"
    
    encoded = tokenizer(
        processed_text,
        padding='max_length',
        truncation=True,
        max_length=20,  # Уменьшаем для наглядности
        return_tensors='pt',
        add_special_tokens=False
    )
    
    token_ids = encoded['input_ids'][0].tolist()
    decoded = tokenizer.decode(token_ids)
    
    print("\nПример токенизации:")
    print(f"Токены: {token_ids}")
    print(f"Декодировано: {decoded}")
    
    try:
        eos_pos = token_ids.index(tokenizer.eos_token_id)
        pad_start = token_ids.index(tokenizer.pad_token_id)
        assert eos_pos < pad_start, "EOS должен быть перед паддингом"
    except ValueError:
        assert False, "EOS token not found"
    
    assert token_ids[0] == tokenizer.bos_token_id
    print("\n✅ Токенизация работает корректно")

def test_masking():
    print("\n" + "="*50)
    print("2. Тестирование создания масок")
    
    batch_size = 2
    seq_len = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    src = torch.ones(batch_size, seq_len, device=device)
    tgt = torch.ones(batch_size, seq_len, device=device)
    
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    
    causal_mask = torch.tril(torch.ones(seq_len, seq_len, device=device))
    tgt_mask = causal_mask.unsqueeze(0).repeat(batch_size, 1, 1, 1)
    
    print("\nSource mask (пример):")
    print(src_mask[0, 0, 0].cpu().numpy()) 
    
    print("\nTarget mask (пример):")
    print(tgt_mask[0, 0].cpu().numpy())
    
    # Проверка размерностей
    assert src_mask.shape == (batch_size, 1, 1, seq_len), f"Ожидалось (2,1,1,5), получено {src_mask.shape}"
    assert tgt_mask.shape == (batch_size, 1, seq_len, seq_len), f"Ожидалось (2,1,5,5), получено {tgt_mask.shape}"
    
    print("\n✅ Маски создаются корректно")

def test_model_forward_pass(model, tokenizer, device):
    print("\n" + "="*50)
    print("3. Тестирование forward pass модели")
    
    test_sentence = "Test input"
    inputs = tokenizer(
        test_sentence,
        return_tensors="pt",
        padding='max_length',
        max_length=128,
        truncation=True
    ).to(device)
    
    src = inputs['input_ids']
    tgt = torch.cat([
        torch.tensor([[tokenizer.bos_token_id]], device=device),
        src[:, :-1]
    ], dim=1)

    src_mask = inputs['attention_mask'].to(torch.bool).unsqueeze(1).unsqueeze(2)
    seq_len = tgt.size(1)
    tgt_mask = torch.tril(torch.ones(seq_len, seq_len, device=device)).to(torch.bool)
    tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        output = model(src, tgt, src_mask, tgt_mask)
    
    print("\nФорма выходов модели:", output.shape)
    print("Минимальное значение:", output.min().item())
    print("Максимальное значение:", output.max().item())
    
    assert not torch.isnan(output).any()
    expected_vocab_size = len(tokenizer)  # Используем реальный размер словаря
    assert output.shape == (1, 128, expected_vocab_size), (
        f"Ожидаемая форма: (1, 128, {expected_vocab_size}), "
        f"получено: {output.shape}"
    )
    print("\n✅ Forward pass работает корректно")

def test_dynamic_masking():
    print("Тестирование динамических масок для разных размеров")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_heads = 8
    d_model = 512
    
    test_cases = [
        {'batch_size': 1, 'seq_len': 7},
        {'batch_size': 3, 'seq_len': 15},
        {'batch_size': 5, 'seq_len': 3}
    ]
    
    for case in test_cases:
        print(f"\nТест для batch={case['batch_size']} seq_len={case['seq_len']}")
        
        src = torch.randint(0, 100, (case['batch_size'], case['seq_len'])).to(device)
        tgt = torch.randint(0, 100, (case['batch_size'], case['seq_len'])).to(device)
        
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = torch.tril(torch.ones(case['seq_len'], case['seq_len'])).to(device)
        tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
        
        mha = MultiHeadAttention(d_model, num_heads).to(device)
        q = k = v = torch.randn(case['batch_size'], case['seq_len'], d_model).to(device)
        
        try:
            output = mha(q, k, v, tgt_mask)
            assert output.shape == (case['batch_size'], case['seq_len'], d_model)
            print("✅ Успех")
        except Exception as e:
            print(f"❌ Ошибка: {str(e)}")
            raise

def test_multi_head_compatibility():
    print("\nТестирование совместимости масок с Multi-Head Attention")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    d_model = 512
    num_heads_list = [4, 8, 16]
    seq_lens = [10, 15, 20]
    
    for num_heads in num_heads_list:
        for seq_len in seq_lens:
            print(f"\nHeads: {num_heads}, Seq_len: {seq_len}")
            
            mask = torch.tril(torch.ones(seq_len, seq_len)).to(device)
            
            mha = MultiHeadAttention(d_model, num_heads).to(device)
            q = torch.randn(2, seq_len, d_model).to(device)
            
            try:
                output = mha(q, q, q, mask)
                assert output.shape == q.shape
                print("✅ Совместимость подтверждена")
            except Exception as e:
                print(f"❌ Несовместимость: {str(e)}")
                raise

def test_translation_function(model, tokenizer, device):
    print("\n" + "="*50)
    print("4. Тестирование функции перевода")
    
    test_sentences = [
        "Hello world",
        "How are you?",
        "Test sentence",
        "Machine learning"
    ]
    
    for sentence in test_sentences:
        print("\n" + "-"*50)
        print(f"Перевод: '{sentence}'")
        
        try:
            translated = translate_sentence(
                model=model,
                tokenizer=tokenizer,
                sentence=sentence,
                device=device,
                max_length=64  # Уменьшенная длина для тестов
            )
            print(f"Результат: {translated}")
        except Exception as e:
            print(f"Ошибка: {str(e)}")
            raise
    
    print("\n✅ Функция перевода работает корректно")

def test_model_with_various_inputs():
    print("\nТестирование модели с различными входными размерами")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    try:
        tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
        tokenizer.add_special_tokens({
            'bos_token': '<s>', 'eos_token': '</s>', 
            'pad_token': '<pad>', 'unk_token': '<unk>'
        })
    except Exception as e:
        print(f"Ошибка инициализации токенизатора: {e}")
        raise
    
    d_model = 512
    num_heads = 8
    num_layers = 6
    
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    ).to(device)
    model.eval()
    
    test_cases = [
        {'batch_size': 1, 'src_len': 10, 'tgt_len': 12},
        {'batch_size': 3, 'src_len': 7, 'tgt_len': 9},
        {'batch_size': 5, 'src_len': 15, 'tgt_len': 15},
        {'batch_size': 2, 'src_len': 5, 'tgt_len': 5}
    ]
    
    for case in test_cases:
        print(f"\nТест: batch={case['batch_size']} src={case['src_len']} tgt={case['tgt_len']}")
        
        try:
            src = torch.randint(0, len(tokenizer), 
                             (case['batch_size'], case['src_len'])).to(device)
            tgt = torch.randint(0, len(tokenizer), 
                             (case['batch_size'], case['tgt_len'])).to(device)
            
            src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(
                (case['batch_size'], case['tgt_len'], case['tgt_len']),
                device=device
            )).bool().unsqueeze(1)
            
            with torch.no_grad():
                output = model(src, tgt, src_mask, tgt_mask)
            
            expected_shape = (
                case['batch_size'], 
                case['tgt_len'], 
                len(tokenizer)
            )
            assert output.shape == expected_shape
            print("✅ Корректная работа")
            
        except Exception as e:
            print(f"❌ Ошибка: {str(e)}")
            raise

def run_comprehensive_tests():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    tokenizer.add_tokens(['<s>', '</s>', '<pad>', '<unk>'])
    tokenizer.bos_token = '<s>'
    tokenizer.eos_token = '</s>'
    tokenizer.pad_token = '<pad>'
    tokenizer.unk_token = '<unk>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<s>')
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('</s>')
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<pad>')
    
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=512,
        num_heads=8,
        num_layers=6
    ).to(device)
    
    test_tokenization(tokenizer)
    test_masking()
    test_model_forward_pass(model, tokenizer, device)
    test_dynamic_masking()
    test_multi_head_compatibility()
    test_model_with_various_inputs()
    test_translation_function(model, tokenizer, device)

run_comprehensive_tests()

1. Тестирование токенизации и специальных токенов
BOS: <s> (id: 62518)
EOS: </s> (id: 0)

Пример токенизации:
Токены: [62518, 160, 5270, 2, 508, 55, 33, 19, 0, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517]
Декодировано: <s>  Hello, how are you?</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

✅ Токенизация работает корректно

2. Тестирование создания масок

Source mask (пример):
[ True  True  True  True  True]

Target mask (пример):
[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1.]]

✅ Маски создаются корректно

3. Тестирование forward pass модели

Форма выходов модели: torch.Size([1, 128, 62519])
Минимальное значение: -2.6944191455841064
Максимальное значение: 2.598710060119629

✅ Forward pass работает корректно
Тестирование динамических масок для разных размеров

Тест для batch=1 seq_len=7
✅ Успех

Тест для batch=3 seq_len=15
✅ Успех

Тест для batch=5 seq_len=3
✅ Успех

Тестирование совместим

## Обучение

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import Dataset

In [21]:
class TranslationDataset(Dataset):
    def __init__(self, processed_data):
        self.input_ids = processed_data['input_ids']
        self.attention_mask = processed_data['attention_mask']
        self.labels = processed_data['labels']
        self.decoder_attention_mask = processed_data['decoder_attention_mask']
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'decoder_attention_mask': torch.tensor(self.decoder_attention_mask[idx], dtype=torch.long)
        }

In [22]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tqdm.auto import tqdm
import wandb
import os
from sklearn.model_selection import train_test_split

In [24]:
import os
import wandb
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [38]:
def calculate_bleu(references, predictions, tokenizer):
    refs = [
        [tokenizer.tokenize(ref.strip(), add_special_tokens=False)]
        for ref in references
    ]
    preds = [
        tokenizer.tokenize(pred.strip(), add_special_tokens=False)
        for pred in predictions
    ]
    smooth = SmoothingFunction().method4
    return corpus_bleu(refs, preds, smoothing_function=smooth)

In [39]:
def create_translation_table(examples_data):
    return wandb.Table(
        columns=["epoch", "step", "source", "target", "prediction"],
        data=examples_data
    )

In [40]:
def evaluate_model(model, val_loader, criterion, tokenizer, device, max_examples=5):
    model.eval()
    total_loss = 0
    all_sources = []
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            src_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(
                labels.size(1)-1, 
                labels.size(1)-1, 
                device=device
            )).bool()
            
            outputs = model(
                input_ids,
                labels[:, :-1],
                src_mask,
                tgt_mask.unsqueeze(0).unsqueeze(0))
            
            loss = criterion(
                outputs.reshape(-1, outputs.size(-1)),
                labels[:, 1:].reshape(-1))
            total_loss += loss.item()
            
            if len(all_sources) < max_examples:
                sources = tokenizer.batch_decode(
                    input_ids, 
                    skip_special_tokens=True
                )
                targets = tokenizer.batch_decode(
                    labels,
                    skip_special_tokens=True
                )
                preds = [
                    translate_sentence(model, tokenizer, src, device)
                    for src in sources
                ]
                all_sources.extend(sources)
                all_targets.extend(targets)
                all_predictions.extend(preds)
    
    bleu_score = calculate_bleu(all_targets, all_predictions, tokenizer)
    
    return (
        total_loss / len(val_loader),
        bleu_score,
        all_sources[:max_examples],
        all_targets[:max_examples],
        all_predictions[:max_examples]
    )

In [41]:
def train_model(subset_size=100, epochs=5, batch_size=32, lr=0.0001, eval_steps=50):
    """Обновленная функция обучения с кумулятивным логированием примеров"""
    config = {
        "group": "baseline-transformer",
        "architecture": "transformer",
        "name": "transformer-base-subset20000-epoch-15-batch8-v1",
        "subset_size": subset_size,
        "epochs": epochs,
        "batch_size": batch_size,
        "learning_rate": lr,
        "eval_steps": eval_steps,
        "d_model": 512,
        "num_heads": 8,
        "num_layers": 6,
    }
    
    run = init_wandb(config=config)
    
    dataset = load_translation_dataset()
    processed_data, tokenizer = prepare_data_with_hf(dataset)
    full_dataset = TranslationDataset(processed_data)
    
    indices = range(min(subset_size, len(full_dataset)))
    subset = torch.utils.data.Subset(full_dataset, indices)
    
    train_size = int(0.7 * len(subset))
    val_size = int(0.15 * len(subset))
    test_size = len(subset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        subset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        **{k: v for k, v in config.items() if k in ['d_model', 'num_heads', 'num_layers']}
    ).to(device)
    
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    
    if run:
        wandb.watch(model, log="all", log_freq=eval_steps)
    
    best_val_loss = float('inf')
    global_step = 0
    cumulative_examples = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            train_loader, 
            desc=f'Epoch {epoch+1}/{epochs}',
            bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}'
        )
        
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            src_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(
                labels.size(1)-1, 
                labels.size(1)-1, 
                device=device
            )).bool()
            
            outputs = model(
                input_ids,
                labels[:, :-1],
                src_mask,
                tgt_mask.unsqueeze(0).unsqueeze(0))
            
            loss = criterion(
                outputs.reshape(-1, outputs.size(-1)),
                labels[:, 1:].reshape(-1))
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            global_step += 1
            
            if global_step % eval_steps == 0:
                val_loss, bleu, sources, targets, preds = evaluate_model(
                    model, val_loader, criterion, tokenizer, device)
                
                # Добавляем новые примеры с метаданными шага и эпохи
                new_entries = [
                    [epoch, global_step, src, tgt, pred]
                    for src, tgt, pred in zip(sources, targets, preds)
                ]
                cumulative_examples.extend(new_entries)
                
                # Создаем обновленную таблицу
                trans_table = create_translation_table(cumulative_examples)
                
                wandb.log({
                    "train/loss": loss.item(),
                    "val/loss": val_loss,
                    "val/bleu": bleu,
                    "examples": trans_table,
                    "epoch": epoch,
                    "step": global_step
                })
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    torch.save(model.state_dict(), "best_model.pth")
                    wandb.save("best_model.pth")
            
            progress_bar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "bleu": f"{bleu:.2f}" if 'bleu' in locals() else "n/a"
            })
        
        avg_loss = total_loss / len(train_loader)
        wandb.log({
            "epoch/train_loss": avg_loss,
            "epoch": epoch
        })
    
    test_loss, test_bleu, test_sources, test_targets, test_preds = evaluate_model(
        model, test_loader, criterion, tokenizer, device, max_examples=10)
    
    test_entries = [
        [epoch, global_step, src, tgt, pred]
        for src, tgt, pred in zip(test_sources, test_targets, test_preds)
    ]
    test_table = create_translation_table(test_entries)
    
    wandb.log({
        "test/loss": test_loss,
        "test/bleu": test_bleu,
        "test_examples": test_table
    })
    
    torch.save(model.state_dict(), "model.pth")
    
    model_artifact = wandb.Artifact("translation-model", type="model")
    model_artifact.add_file("model.pth")
    wandb.log_artifact(model_artifact)
    
    wandb.finish()
    
    return model, best_val_loss, test_loss

In [42]:
model, best_val_loss, test_loss = train_model(
    subset_size=20000,
    epochs=15,
    batch_size=8,
    lr=0.0001,
    eval_steps=100
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


✅ W&B successfully initialized
Loading Tatoeba en-ru...

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

Data sample:
EN: For once in my life I'm doing a good deed... And it is useless.
RU: Один раз в жизни я делаю хорошее дело... И оно бесполезно.

EN: Let's try something.
RU: Давайте что-нибудь попробуем!



Epoch 1/15:  19%|█▉        | 99/525 [00:22<01:38,  4.33it/s, loss=2.8641, bleu=n/a]Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'add_special_tokens': False} not recognized.
Keyword arguments {'a

KeyboardInterrupt: 